In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


In [4]:
file_path = "jumlah-peserta-didik-sd-2024-sd-mi-sederajat.xlsx"
df_raw = pd.read_excel(file_path)

print("Data Asli (5 baris pertama):")
print(df_raw.head())

Data Asli (5 baris pertama):
  Jumlah Peserta Didik Sekolah Dasar (SD) berdasarkan Wilayah dan Status Satuan Pendidikan, Tahun 2023  \
0                                                NaN                                                     
1                                                 No                                                     
2                                                NaN                                                     
3                                                  1                                                     
4                                                  2                                                     

      Unnamed: 1 Unnamed: 2      Unnamed: 3                Unnamed: 4  \
0            NaN        NaN             NaN                       NaN   
1  Kode Kemdagri   Kode BPS        Provinsi  Status Satuan Pendidikan   
2            NaN        NaN             NaN                    Negeri   
3             11         11            Ace

In [5]:
df = df_raw.iloc[3:].reset_index(drop=True)
df.columns = ["No", "Kode_Kemdagri", "Kode_BPS", "Provinsi", "Negeri",
              "Swasta", "Total"]

# Konversi ke numerik
num_cols = ["No", "Kode_Kemdagri", "Kode_BPS", "Negeri", "Swasta", "Total"]
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")


In [6]:
df = df.dropna(subset=["Provinsi"]).reset_index(drop=True)

print("\nDataset Setelah Dibersihkan:")
print(df.head())


Dataset Setelah Dibersihkan:
    No  Kode_Kemdagri  Kode_BPS        Provinsi   Negeri  Swasta    Total
0  1.0           11.0      11.0            Aceh   451761   38102   489863
1  2.0           12.0      12.0  Sumatera Utara  1177930  346551  1524481
2  3.0           13.0      13.0  Sumatera Barat   521822   70543   592365
3  4.0           14.0      14.0            Riau   633978  142596   776574
4  5.0           15.0      15.0           Jambi   333373   35247   368620


In [7]:
print("\nCek Missing Value:")
print(df.isnull().sum())



Cek Missing Value:
No               1
Kode_Kemdagri    2
Kode_BPS         2
Provinsi         0
Negeri           0
Swasta           0
Total            0
dtype: int64


In [8]:
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

print("\nMissing value setelah imputasi:")
print(df.isnull().sum())


Missing value setelah imputasi:
No               0
Kode_Kemdagri    0
Kode_BPS         0
Provinsi         0
Negeri           0
Swasta           0
Total            0
dtype: int64


In [9]:
for col in ["Negeri", "Swasta", "Total"]:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = np.clip(df[col], lower, upper)


In [10]:
scaler = StandardScaler()
scale_cols = ["Negeri", "Swasta", "Total"]

df[scale_cols] = scaler.fit_transform(df[scale_cols])


In [11]:
# 1) Fitur Rasio Swasta
df["swasta_ratio"] = df["Swasta"] / (df["Swasta"] + df["Negeri"])

# 2) Fitur Rasio Negeri
df["negeri_ratio"] = df["Negeri"] / (df["Swasta"] + df["Negeri"])

# 3) Kategori ukuran provinsi
df["kategori_provinsi"] = pd.cut(
    df["Total"],
    bins=[0, 500000, 1000000, 2000000],
    labels=["Kecil", "Sedang", "Besar"]
)

# 4) Fitur interaksi (Negeri x Swasta)
df["interaksi_ns"] = df["Negeri"] * df["Swasta"]

# 5) Fitur kepadatan siswa (Total / 2 variabel kode)
df["density_score"] = df["Total"] / (df["Kode_Kemdagri"] + df["Kode_BPS"])

In [12]:
new_num_cols = ["swasta_ratio", "negeri_ratio", "interaksi_ns", "density_score"]

df[new_num_cols] = scaler.fit_transform(df[new_num_cols])

In [13]:
print("\nStruktur dataset final:")
print(df.info())

print("\n5 baris dataset final:")
print(df.head())


Struktur dataset final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   No                 40 non-null     float64 
 1   Kode_Kemdagri      40 non-null     float64 
 2   Kode_BPS           40 non-null     float64 
 3   Provinsi           40 non-null     object  
 4   Negeri             40 non-null     float64 
 5   Swasta             40 non-null     float64 
 6   Total              40 non-null     float64 
 7   swasta_ratio       40 non-null     float64 
 8   negeri_ratio       40 non-null     float64 
 9   kategori_provinsi  14 non-null     category
 10  interaksi_ns       40 non-null     float64 
 11  density_score      40 non-null     float64 
dtypes: category(1), float64(10), object(1)
memory usage: 3.7+ KB
None

5 baris dataset final:
    No  Kode_Kemdagri  Kode_BPS        Provinsi    Negeri    Swasta     Total  \
0  1.0    

In [14]:
df.to_excel("dataset_pendidikan_final.xlsx", index=False)
print("\nDataset final berhasil disimpan sebagai dataset_pendidikan_final.xlsx")


Dataset final berhasil disimpan sebagai dataset_pendidikan_final.xlsx
